# REINFORCE Algorithm

### 1. Muestrar {$\tau^i$} de $\pi_{\theta}(a_t|s_t)$ - Correr M trayectorias usando la policy
### 2. Estimar el retorno: $$ R(\tau_i)  \approx \sum_{t=0}^{T}R(s_t^i, a_t^i)$$
### 3. Entrenar un modelo: $$ \nabla_{\theta} J_{\theta} \approx \frac{1}{M} \sum_{i=1}^{M}  R(\tau_i)   \sum_{t=0}^T \nabla_{\theta} log \pi_{\theta}(a_t^i|s_t^i)$$

# Implementación en Keras

### Suponiendo que solo corremos un episodio por iteración

La loss queda (Le ponemos un menos adelante para que tengamos que minimizar):

$$ \huge J_{\theta} =  - R \sum_{t=0}^T log \pi_{\theta}(a_t|s_t)$$
$$ \huge J_{\theta} =  - \sum_{t=0}^T log \pi_{\theta}(a_t|s_t) R $$
$$ \huge J_{\theta} =  \sum_{t=0}^T log \frac{1}{\pi_{\theta}(a_t|s_t)} R $$

Recordando la Entropía cruzada:

$$ \huge H(y_{true}, y_{pred}) = \sum_{i} y_{true_i} log (\frac{1}{y_{pred_i}}) $$

Ejemplo: 

- Sumpongamos que tenemos 3 acciones posibles y la red neuronal predijo $y_{pred}$ = [0.2, 0.3, 0.5]
- Se muestreó la salida y se eligión la acción 2, es decir la acción con probabilidad 0.3
- La $y_{true}$ será [0, 1, 0]

$$ \huge H = 0 log (\frac{1}{0.2}) + 1 log (\frac{1}{0.3}) + 0 log (\frac{1}{0.5}) $$

- Si redefinimos la $y_{true}$ como $y_{true}$ = $y_{true} R$
- La $y_{true}$ queda [0, R, 0]

$$ \huge H = 0 log (\frac{1}{0.2}) + R log (\frac{1}{0.3}) + 0 log (\frac{1}{0.5}) = R log (\frac{1}{\pi_{\theta}(a_t|s_t)}) $$

In [29]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
# conda install -c conda-forge tensorboardx

In [31]:
from REINFORCE_helper import BaseAgent, format_as_pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam, SGD
import keras.backend as K
import numpy as np

In [32]:
class ReinforceAgent(BaseAgent):
    # def __init__(self):
    def get_policy_model(self, lr=0.001, hidden_layer_neurons = 128, input_shape=[4], output_shape=2):
        ## Defino métrica - loss sin el retorno multiplicando
        def loss_metric(y_true, y_pred):
            y_true_norm = K.sign(y_true)
            return K.categorical_crossentropy(y_true_norm, y_pred)
        model = Sequential()
        model.add(Dense(hidden_layer_neurons, input_shape=input_shape, activation='relu'))
        model.add(Dense(output_shape, activation='softmax'))
        ## Por que la categorical_crossentropy funciona ok?
        model.compile(Adam(lr), loss=['categorical_crossentropy'], metrics=[loss_metric])
        return model
    
    def get_action(self, eval=False):
        p = self.model.predict([self.observation.reshape(1, self.nS)])
        if eval is False:
            action = np.random.choice(self.nA, p=p[0]) #np.nan_to_num(p[0])
        else:
            action = np.argmax(p[0])
        action_one_hot = np.zeros(self.nA)
        action_one_hot[action] = 1
        return action, action_one_hot, p
    
    def get_entropy(self, preds, epsilon=1e-12):
        entropy = np.mean(-np.sum(np.log(preds+epsilon)*preds, axis=1)/np.log(self.nA))
        return entropy
    
    def get_discounted_rewards(self, r):
        # Por si es una lista
        r = np.array(r, dtype=float)
        """Take 1D float array of rewards and compute discounted reward """
        discounted_r = np.zeros_like(r)
        running_add = 0
        for t in reversed(range(0, r.size)):
            running_add = running_add * self.gamma + r[t]
            discounted_r[t] = running_add
        return discounted_r 

In [33]:
reinforce_agent = ReinforceAgent('CartPole-v1', n_experience_episodes=3)

# reinforce_agent.logdir

In [34]:
# reinforce_agent.model.summary()

In [35]:
reinforce_agent.reset_env()
action, action_one_hot, p = reinforce_agent.get_action()
print('Action:', action)
print('action_one_hot:', action_one_hot)
print('Policy prob dist:', p)

Action: 1
action_one_hot: [0. 1.]
Policy prob dist: [[0.4994691  0.50053084]]


In [36]:
reinforce_agent.reset_env()
obs, actions, preds, disc_sum_rews, rewards, ep_returns, ep_len, last_obs, time_step = reinforce_agent.get_experience_episodes(return_ts=True)

In [37]:
format_as_pandas(time_step, obs, preds, actions, rewards, disc_sum_rews, ep_returns, decimals = 3)

,step,observation,policy_distribution,sampled_action,rewards,discounted_sum_rewards,episode_return
0,1,"[-0.041, -0.01, 0.004, 0.022]","[0.501, 0.498]","[0, 1]",1.0,14.895,14.895
1,2,"[-0.041, 0.184, 0.004, -0.268]","[0.5, 0.499]","[0, 1]",1.0,13.909,14.895
2,3,"[-0.037, 0.379, 0.0, -0.559]","[0.498, 0.501]","[0, 1]",1.0,12.922,14.895
3,4,"[-0.03, 0.574, -0.011, -0.852]","[0.496, 0.503]","[1, 0]",1.0,11.934,14.895
4,5,"[-0.018, 0.379, -0.028, -0.563]","[0.497, 0.502]","[1, 0]",1.0,10.945,14.895
5,6,"[-0.011, 0.184, -0.04, -0.28]","[0.498, 0.501]","[0, 1]",1.0,9.955,14.895
6,7,"[-0.007, 0.38, -0.045, -0.585]","[0.496, 0.503]","[1, 0]",1.0,8.964,14.895
7,8,"[0.0, 0.185, -0.057, -0.307]","[0.497, 0.502]","[0, 1]",1.0,7.972,14.895
8,9,"[0.003, 0.381, -0.063, -0.617]","[0.494, 0.505]","[0, 1]",1.0,6.979,14.895
9,10,"[0.011, 0.577, -0.075, -0.929]","[0.492, 0.507]","[0, 1]",1.0,5.985,14.895


In [38]:
reinforce_agent.get_entropy(preds)

0.99932164

# Algoritmo REINFORCE

In [39]:
from REINFORCE_helper import RunningVariance
from time import time

# Pruebas interesantes:
- n_experience_episodes=1, epochs=1, lr=0.001
- n_experience_episodes=5, epochs=1, lr=0.001
- n_experience_episodes=50, epochs=1, lr=0.001
- n_experience_episodes=50, epochs=20, lr=0.001
- n_experience_episodes=50, epochs=50, lr=0.001
- n_experience_episodes=50, epochs=50, lr=0.01

In [47]:
running_variance.get_mean()

62.69705640189479

In [46]:
reinforce_agent = ReinforceAgent('CartPole-v1', n_experience_episodes=5, EPISODES=2000, 
                                 epochs=1, lr=0.1, gamma=0.999)
# reinforce_agent = ReinforceAgent('LunarLander-v2', n_experience_episodes=1, EPISODES=2000, epochs=1, lr=0.001)

running_variance = RunningVariance()
initial_time = time()


while reinforce_agent.episode < reinforce_agent.EPISODES:
    obs, actions, preds, disc_sum_rews, rewards, ep_returns, ep_len, last_obs = reinforce_agent.get_experience_episodes()
    for dr in ep_returns:
        running_variance.add(dr)
        
    pseudolabels = actions*ep_returns.reshape(-1, 1)

    history = reinforce_agent.model.fit(obs, pseudolabels, verbose=0, epochs=reinforce_agent.epochs, batch_size=128)
    
    reinforce_agent.log_data(reinforce_agent.episode, 
                      history.history['loss'][0], 
                      np.mean(ep_len), 
                      reinforce_agent.get_entropy(preds), 
                      running_variance.get_variance(), 
                      history.history['loss_metric'][0], 
                      time() - initial_time, np.mean(ep_returns[-1]))

correr en linea de comando: tensorboard --logdir logs/
Episode: 51
Model on episode 52 improved from -inf to 41.15036891544908. Saved!
Episode: 103
Model on episode 104 improved from 41.15036891544908 to 45.935038158264014. Saved!
Episode: 155
Model on episode 156 improved from 45.935038158264014 to 57.321058835667465. Saved!
Episode: 467
Model on episode 468 improved from 57.321058835667465 to 75.07167043185704. Saved!
Episode: 519
Model on episode 520 improved from 75.07167043185704 to 154.7172194264973. Saved!
Episode: 2002